In [1]:
import pandas as pd
import numpy as np

class SME():
    
    def __init__(self):
        self.asked = 0
        self.df = self.get()
        
    def get(self):
        X_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_X.csv')
        y_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_y.csv')

        X_train['Left'] = y_train
        return X_train
    
    def ask(self, valuedict):
        self.asked += 1
        #  print(self.asked)
        
        if self.asked > 500:
            raise Exception("Sorry, you have asked enough")

        arr = []
        for prop in valuedict:
            val = valuedict[prop]

            if val is None: 
                continue

            val = f"'{val}'" if isinstance(val,str) else val
            # print(prop, val)
            arr.append(f'{prop} == {val}')

            query = ' and '.join(arr)
        result = self.df.query(query)

        if len(result) == 0:
            raise Exception("I don't know")
        
        return (int)(result['Left'].mean())

In [2]:
sme = SME()

In [3]:
sme.df.columns

Index(['RecordId', 'Gender', 'Distance', 'YearsWorked', 'TrainingHours',
       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',
       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',
       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',
       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',
       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',
       'DepartmentCode', 'PreviousSalary', 'Salary', 'SelfReview',
       'SupervisorReview', 'Left'],
      dtype='object')

In [4]:
X_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_X.csv')

In [5]:

sme.df['SelfReview'] = sme.df['SelfReview'].fillna(sme.df['SelfReview'].median())
sme.df['SupervisorReview'] = sme.df['SupervisorReview'].fillna(sme.df['SupervisorReview'].median())
sme.df[['Salary', 'PreviousSalary']] = sme.df[['Salary', 'PreviousSalary']].map(lambda x: x.replace('K', '000') if isinstance(x, str) else x)
sme.df[['Salary', 'PreviousSalary']] = sme.df[['Salary', 'PreviousSalary']].astype(int)

sme.df['Raise'] = (sme.df['Salary'] / sme.df['PreviousSalary']) - 1
sme.df['ReviewDiff'] = sme.df['SelfReview'] - sme.df['SupervisorReview']
sme.df['ReviewOverRaise'] = sme.df['SelfReview'] / (sme.df['Raise'] +.0001)

sme.df['LongDist'] = np.where(sme.df['Distance'].isin(['~20miles', '>30miles']), 1, 0)

sme.df['HighStress'] = np.where(sme.df['StressLevel'] > 3.5, 1, 0)

num = ['HighStress', 'LongDist','Gender', 'SupervisorReview','SelfReview', 'Raise', 'Salary','ReviewOverRaise', 'PreviousSalary'] #, 'LongDist', ReviewOverRaise','ReviewDiff', 'StressLevel'
cat = ['DepartmentCode']#'Distance',
#num = ['Raise', 'StressLevel']

"""
num = ['HighStress', 'LongDist','Gender','RecordId', 'YearsWorked', 'TrainingHours', 
       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',
       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',
       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',
       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',
       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',
       'PreviousSalary', 'Salary', 'SelfReview',
       'SupervisorReview', 'Raise', 'ReviewDiff', 'ReviewOverRaise']
"""

"\nnum = ['HighStress', 'LongDist','Gender','RecordId', 'YearsWorked', 'TrainingHours', \n       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',\n       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',\n       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',\n       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',\n       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',\n       'PreviousSalary', 'Salary', 'SelfReview',\n       'SupervisorReview', 'Raise', 'ReviewDiff', 'ReviewOverRaise']\n"

In [6]:
sme.df[num].corr()>.1

,HighStress,LongDist,Gender,SupervisorReview,SelfReview,Raise,Salary,ReviewOverRaise,PreviousSalary
HighStress,True,False,False,False,False,False,False,False,False
LongDist,False,True,False,False,False,False,False,False,False
Gender,False,False,True,False,False,False,False,False,False
SupervisorReview,False,False,False,True,False,False,False,False,False
SelfReview,False,False,False,False,True,False,False,False,False
Raise,False,False,False,False,False,True,False,False,False
Salary,False,False,False,False,False,False,True,False,True
ReviewOverRaise,False,False,False,False,False,False,False,True,False
PreviousSalary,False,False,False,False,False,False,True,False,True


# Code for Feature Engineering

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEngineering(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X[['Salary', 'PreviousSalary']] = X[['Salary', 'PreviousSalary']].map(lambda x: x.replace('K', '000') if isinstance(x, str) else x)
        X[['Salary', 'PreviousSalary']] = X[['Salary', 'PreviousSalary']].astype(int)
        X['Raise'] = (X['Salary'] / X['PreviousSalary']) - 1
        X['ReviewDiff'] = X['SelfReview'] - X['SupervisorReview']
        X['ReviewOverRaise'] = X['SelfReview'] / (X['Raise'] +.0001)
        return X

# Establish the preprocessing for numeric and categorical features.

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans

n_clusters = 500

featurepreprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num)#,
        #('cat', OneHotEncoder(), cat)
    ])
preprocessor = Pipeline(steps=[
    #('featureengineering', FeatureEngineering()),
    ('features', featurepreprocessor),
    ('pca', PCA(n_components=.9)),#.9
    ('kmeans', KMeans(n_clusters=n_clusters, random_state=42, n_init='auto'))
    ])
#('kmeans', KMeans(n_clusters=n_clusters, n_init='auto'), num)#+cat)
#('kmeans', KMeans(n_clusters=n_clusters, n_init='auto')

In [11]:
#X_dist = preprocessor.fit_transform(sme.df[num+cat])

In [12]:
preprocessor

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['HighStress', 'LongDist',
                                                   'Gender', 'SupervisorReview',
                                                   'SelfReview', 'Raise',
                                                   'Salary', 'ReviewOverRaise',
                                                   'PreviousSalary'])])),
                ('pca', PCA(n_components=0.9)),
                ('kmeans',
                 KMeans(n_clusters=500, n_init='auto', random_state=42))])

In [13]:
kmeans = KMeans(n_clusters = n_clusters, n_init='auto')
pre = featurepreprocessor.fit_transform(sme.df[num])#+cat])

In [14]:
X_dist = kmeans.fit_transform(pre)
X_dist

array([[2.82928005, 4.38492437, 3.04445243, ..., 2.76823996, 6.70963652,
        3.69610989],
       [4.04254793, 2.82169483, 3.60244443, ..., 4.39124698, 6.97777481,
        3.25646364],
       [4.15861716, 3.92049174, 2.98719087, ..., 3.82743188, 6.53524018,
        2.93242188],
       ...,
       [4.28048533, 3.46325808, 3.802212  , ..., 3.02658851, 6.61307093,
        4.44829224],
       [4.50990804, 1.71050271, 3.17831817, ..., 4.05300041, 6.85383275,
        5.13820163],
       [3.83602795, 5.2850494 , 4.20703148, ..., 5.25471839, 8.20782449,
        3.01063348]])

In [15]:
X_dist.shape

(500000, 500)

In [16]:
#How many points are in each cluster?
pd.Series(kmeans.labels_).value_counts()

260    3574
103    3280
232    2971
66     2939
173    2938
       ... 
287     212
462     192
459     188
362     186
119     142
Name: count, Length: 500, dtype: int64

In [17]:
import numpy as np
#What are the reporesentative points? (e.i the ones closest to each centroid)
representative_idx = np.argmin(X_dist, axis=0)
representative_idx.shape

(500,)

In [18]:
#What is the index number of each representative point?
representative_idx

array([ 18125,   2018,   2197,    235,  27973,   6026,   5425,  10140,
         4795,   7742,    339,    353,     41,  11370,   1252,  33565,
          308,  59257,   3506,   1005,   2934,   7475,  13114,   5795,
        25679,   2792,   9159,   3154,   1485,   2030,   5788,  57186,
         8445,   7988,   2149,   2732,   8563,  24857,     12,   5758,
          806,   6751,   1199,   6386,   5091,   7067, 212600,  17078,
          459,    498,   7091,   3992,   5535,   3291,  15088,  28388,
         6096,   2472,   4566,   2671,  14391,  10124,   2048,   3803,
         1157,   2828,   2322,   6395,    885,   9406,  18808,  52921,
        17465,   3413,  69916,  32151,   1628,  20715,  12000,  26465,
        15570,   3567,   4185, 103970,   1152,   5847,   7184,  24668,
         1882,   3767,  22013,  23860,    512,  21656,   6718,   5940,
        36955,  45016,   2714, 185504,   1956,     85,  13441,   1988,
        16081,   1492,   1062,   5011,    263,  28668,   7422,  22490,
      

In [19]:
#What do the reporesentative points look like?
X_representative = sme.df.iloc[representative_idx]
X_representative

,RecordId,Gender,Distance,YearsWorked,TrainingHours,WorkLifeBalance,NumOfProjects,JobInvolvement,TeamSize,MentorshipReceived,...,PreviousSalary,Salary,SelfReview,SupervisorReview,Left,Raise,ReviewDiff,ReviewOverRaise,LongDist,HighStress
18125,18225,0,>30miles,7,90,3,8,2,6,1,...,83000,85000,4.0,5.0,1,0.024096,-1.0,165.313947,1,0
2018,2118,0,~10miles,10,24,3,10,3,12,1,...,52000,54000,5.0,4.0,0,0.038462,1.0,129.662877,0,0
2197,2297,1,~10miles,5,98,3,4,1,16,0,...,63000,64000,5.0,3.0,1,0.015873,2.0,313.027924,0,0
235,335,0,~10miles,6,60,5,15,1,2,1,...,58000,59000,4.0,4.0,1,0.017241,0.0,230.662159,0,1
27973,28073,0,~10miles,5,48,4,1,3,4,0,...,54000,54000,4.0,4.0,1,0.000000,0.0,40000.000000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1985,0,~15miles,7,67,1,3,4,6,0,...,59000,60000,5.0,5.0,0,0.016949,0.0,293.269709,0,0
7050,7150,0,~15miles,5,78,2,5,5,5,0,...,55000,58000,5.0,5.0,0,0.054545,0.0,91.498919,0,1
20384,20484,1,~20miles,7,46,5,15,4,1,1,...,58000,59000,5.0,2.0,1,0.017241,3.0,288.327699,1,0
267684,267784,0,~20miles,11,38,2,7,4,7,0,...,56000,56000,5.0,2.0,1,0.000000,3.0,50000.000000,1,0


In [20]:
#Look up the leave/stay value of hte 500 representative points
y_representative= []
for i in representative_idx:
    y_representative.append(sme.ask({'RecordId': i+100})) #All record Id's are 100 more than the index value

In [21]:
#Propagate the label value to each point in the cluster.
y_train_propagated = np.empty(len(sme.df), dtype=np.int32)
for i in range(n_clusters):
    y_train_propagated[kmeans.labels_ == i] = y_representative[i]

In [22]:
#Assign the labels and leave/stay values to the DF
sme.df['Labels'] = kmeans.labels_
sme.df['y_pred'] = y_train_propagated

In [23]:
X_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_X.csv')
X_train['Left'] = y_train_propagated
X_train.columns

Index(['RecordId', 'Gender', 'Distance', 'YearsWorked', 'TrainingHours',
       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',
       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',
       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',
       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',
       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',
       'DepartmentCode', 'PreviousSalary', 'Salary', 'SelfReview',
       'SupervisorReview', 'Left'],
      dtype='object')

In [24]:
sme.df

,RecordId,Gender,Distance,YearsWorked,TrainingHours,WorkLifeBalance,NumOfProjects,JobInvolvement,TeamSize,MentorshipReceived,...,SelfReview,SupervisorReview,Left,Raise,ReviewDiff,ReviewOverRaise,LongDist,HighStress,Labels,y_pred
0,100,1,>30miles,7,91,4,14,3,8,1,...,4.0,4.0,0,0.013889,0.0,285.941223,1,0,337,0
1,101,0,~10miles,10,52,5,9,2,10,0,...,4.0,4.0,0,0.032258,0.0,123.616788,0,1,180,0
2,102,0,~10miles,10,80,3,13,3,16,0,...,4.0,2.0,1,0.013699,2.0,289.883848,0,0,137,1
3,103,0,~15miles,5,6,3,15,5,16,0,...,5.0,5.0,0,0.015873,0.0,313.027924,0,0,263,0
4,104,1,~10miles,10,3,2,13,3,18,0,...,4.0,4.0,0,0.066667,0.0,59.910135,0,0,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,500095,1,>30miles,9,80,1,10,3,14,0,...,4.0,5.0,0,0.032787,-1.0,121.629031,1,1,308,0
499996,500096,0,~20miles,7,0,3,2,5,4,1,...,5.0,5.0,0,0.033333,0.0,149.551346,1,1,290,0
499997,500097,0,~20miles,10,8,3,13,3,13,1,...,5.0,2.0,0,0.046154,3.0,108.099119,1,0,403,0
499998,500098,0,~15miles,10,51,3,7,3,5,0,...,5.0,5.0,1,0.018868,0.0,263.602905,0,0,48,0


In [25]:
sme.df['ClusteringOff']= abs(sme.df['y_pred'] - sme.df['Left']) #Which points are incorrectly labeled / is the clustering off for?

In [26]:
#What Percent incorrect is each cluster?
percent_df = sme.df.groupby('Labels')['ClusteringOff'].sum()/sme.df.groupby('Labels')['ClusteringOff'].count()
percent_df
#percent_df[percent_df>.5].count()

Labels
0      0.473398
1      0.548791
2      0.747352
3      0.053769
4      0.036090
         ...   
495    0.122878
496    0.061674
497    0.510507
498    0.132867
499    0.271028
Name: ClusteringOff, Length: 500, dtype: float64

In [27]:
#What is the total accuracy of the model?
1- sme.df['ClusteringOff'].sum() / sme.df['ClusteringOff'].count()

0.755126

In [28]:
#What percentage of "Leaving" predictions did our model get wrong?
sme.df[sme.df['Left']==1]['ClusteringOff'].sum() / sme.df[sme.df['Left']==1]['ClusteringOff'].count()

0.34178021844054945

In [29]:
sme.df[sme.df['Left']==0]['ClusteringOff'].sum() / sme.df[sme.df['Left']==0]['ClusteringOff'].count()

0.19014580379859736

In [30]:
#How many "Stays" should be "leaves?
sme.df[sme.df['Left']==1]['ClusteringOff'].sum()

61678

In [31]:
#How many "leaves" Should be "stays"?
sme.df[sme.df['Left']==0]['ClusteringOff'].sum()

60759

# Cleaning Pipeline to set up for ML